In [ ]:
#default_exp snowflake.query

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_azure.ipynb.
Converted 02_utils_parseyaml.ipynb.
Converted 03_utils_dataframes.ipynb.
Converted 04_dstools_preparedata.ipynb.
Converted 05_snowflake_query.ipynb.
Converted 06_snowflake_copyinto.ipynb.
Converted index.ipynb.


# ``SnowflakeTool:``

In [ ]:
#export
from easymigration.imports import *
from easymigration.utils.parseyaml import *
from easymigration import files

In [ ]:
#export
class SnowflakeTool:
    """
    Class that holds basic snowflake functionality including testing connection
    and running queries.
    """

    def __init__(self,
                 sfAccount: str = None,
                 sfUser: str = None,
                 sfPswd: str = None,
                 sfWarehouse: str = None,
                 sfDatabase: str = None,
                 sfSchema: str = None,
                 sfRole: str = None,
                 delimiter: str = ';',
                 logger = None
                 ):
        """
        Creates a connection to snowflake account for a simple class
        for a python users to take advantage of the computation power of
        snowflake. This class is used in all snowflake .py files through
        super().

        Args:
            sfAccount (str, optional): snowflake credential passed as string. Defaults to None.
            sfUser (str, optional): snowflake credential passed as string. Defaults to None.
            sfPswd (str, optional): snowflake credential passed as string. Defaults to None.
            sfWarehouse (str, optional): snowflake credential passed as string. Defaults to None.
            sfDatabase (str, optional): snowflake credential passed as string. Defaults to None.
            sfSchema (str, optional): snowflake credential passed as string. Defaults to None.
            sfRole (str, optional): snowflake credential passed as string. Defaults to None.
            delimiter (str, optional): parse delimiter for the query files. Defaults to ';'.
            logger ([type], optional): pass custom logger as many libraries are set to Warning. Defaults to None.
        """
        self.engine = create_engine(URL(user=sfUser,
                                        password=sfPswd,
                                        account=sfAccount,
                                        warehouse=sfWarehouse,
                                        database=sfDatabase,
                                        schema=sfSchema,
                                        role=sfRole))
        self._logger = logger if logger is not None else logging.getLogger(__name__)
        self.delimiter = delimiter
        self._logger.info("sqlalchemy snowflake engine created")

    @staticmethod
    def parse_sql_file(path: str, delimiter: str):
        """
        Helper function to parse string on default delimiter ;
        Args:
            path (str): path to file that will be parsed
            delimiter (str): string type to parse end of sql command

        Returns:
            [type]: [description]
        """

        with open(path, 'r') as file:
            lines = file.readlines()

        single_command = ''
        sql_commands = []

        for i in lines:
            single_command = single_command + i
            if f'{delimiter}' in i:
                sql_commands.append(single_command)
                single_command = ''

        return sql_commands

    def run_str_query(self, query: str):
        """
        This function will connect to snowflake and run a query that is passed
        in as a string.
        Args:
            query (str): SQL Query String
        Returns:
            pd.DataFrame/ None: dependent on the query string
        """

        with self.engine.connect() as connection:
            try:
                if connection:
                    df = pd.read_sql_query(query, self.engine)
                    self._logger.info("data loaded from snowflake")
                else:
                    self._logger.info("connection to snowflake failed")
            finally:
                self.engine.dispose()
                connection.close()
                self._logger.info("connection to snowflake has been turned off")
        if df.columns[0] == 'status':
            self._logger.info(df.status.values[0])
        else:
            return df

    def execute_file_query(self, query_path: str):
        """
        Same as ``excecute file``, but with no sql option
        might not need this.

        Args:
            query_path (str): file location to execute
        """

        self._logger.info("query being parsed")
        sql_commands = self.parse_sql_file(query_path, self.delimiter)

        with self.engine.connect() as connection:
            try:
                if connection:
                    for i in sql_commands:
                        connection.execute(i)
                    self._logger.info("data loaded from snowflake")
                else:
                    self._logger.info("connection to snowflake failed")
            finally:
                self.engine.dispose()
                connection.close()
                self._logger.info("connection to snowflake has been turned off")

    def execute_file(self, sql: str = None, query_path: str = None):
        """
        will run sql file or txt file that is seperated by self.delimenter
        with the option to add a custom sql command if there is something you
        want to add to the file being executed

        Args:
            sql (str, optional): Optional Command to add on top of the file being executed. Defaults to None.
            query_path (str, optional): location to sql/txt file to execute. Defaults to None.

        Returns:
            pd.DataFrame: returns query results
        """

        self._logger.info("query being parsed")
        sql_commands = self.parse_sql_file(query_path, self.delimiter)

        with self.engine.connect() as connection:
            try:
                if connection:
                    for i in sql_commands:
                        connection.execute(i)
                        self._logger.info("sql or txt file excecuted")
                    if sql:
                        df = pd.read_sql_query(sql, self.engine)
                        self._logger.info("data is loaded from snowflake")
                else:
                    self._logger.info("connection to snowflake failed")
                    df = None
                return df

            finally:
                self.engine.dispose()
                connection.close()
                self._logger.info("connection to snowflake has been turned off")
                
    def pandas_to_snowflake(self,
                            df: 'pd.DataFrame',
                            user: str,
                            password: str,
                            account: str,
                            warehouse: str,
                            database: str,
                            schema: str,
                            table_name: str,
                            chunk_size: int = None,
                            create_table: bool = False,
                            create_statement: str = None,
                            parallel: int = 4):
        """
        Function that expect a table has been created as the infer function mis
        allocates the size of columns and isn't best practice not chooses the dytpes
        of a column, but is much easier to call upon ``infer_to_snowflake`` recommended
        when in development.
        Args:
            df (pd.DataFrame): Dataframe that is going to be sent to snowflake
            user (str): snowflake credential passed as string
            password (str): snowflake credential passed as string
            account (str): snowflake credential passed as string
            warehouse (str): snowflake credential passed as string
            database (str): snowflake credential passed as string
            schema (str): snowflake credential passed as string
            table_name (str): Table name being created inside of snowflake
            chunk_size (int, optional): Number of elements to be inserted once, if not provided all elements will be dumped once.
                                            Defaults to None.
            create_table (bool, optional): If true will create a table with create_statement. Defaults to False.
            create_statement (str, optional): Give create table statement and the table will
                                                be created from this string. Defaults to None.
            parallel (int, optional): Number of threads to be used when uploading chunks. Defaults to None.                                                
        """
        _connection = connector.connect(user=os.environ['sfUser'],
                                        password=os.environ['sfPswd'],
                                        account=os.environ['sfAccount'],
                                        warehouse=os.environ['sfWarehouse'],
                                        database=os.environ['sfDatabase'],
                                        schema=os.environ['sfSchema'])
        if create_table:
            self.run_str_query(create_statement)
        insert_areas = ['INSERT_TABLE_NAME_HERE', 'INSERET_DATABASE_HERE', 'INSERT_SCHEMA_HERE']
        inserts = [table_name, database, schema]
        table_check = ParseYaml(os.path.join(os.path.abspath(files.__path__[0]), 'snowflake.yaml')).get_yaml(['test']).get('check_table')
        for k, v in zip(insert_areas, inserts):
            table_check = table_check.replace(k, v)
        check = self.run_str_query(table_check)
        if check.shape[0] == 0:
            raise
        try:
            df.columns = [x.upper() for x in df.columns]
            response, nchunks, nrows, _ = write_pandas(conn=_connection, df=df, table_name=table_name.upper(),
                                                       chunk_size=chunk_size, parallel=parallel)
            self._logger.info(f'Upload was a success: {response} & the number of rows loaded {nrows}')
            if nrows != df.shape[1]:
                self._logger.warning(f'Dataframe failed to load {nrows - df.shape[1]}')
        except Exception as e:
            self._logger.error(f'{e}')
            raise
        self._logger.info(f'Dataframe uploaded to {database}{table_name}')

    def infer_to_snowflake(self,
                           df: 'pd.DataFrame',
                           table_name: str,
                           if_exists: str = 'fail',
                           dtype: dict = None,
                           chunk_size: int = None):
        """
        Args:
            df (pd.DataFrame): The dataframe that is going to snowflake
            table_name (str): Table name that will be created in snowflake
            if_exists (str, optional): How to behave if the table already exists.
                                        * fail: Raise a ValueError.
                                        * replace: Drop the table before inserting new values.
                                        * append: Insert new values to the existing table.
                                        Defaults to 'fail'.
            dtype (dict, optional): Specifying the datatype for columns. If a dictionary is used, the
                                    keys should be the column names and the values should be the
                                    SQLAlchemy types or strings for the sqlite3 legacy mode. If a
                                    scalar is provided, it will be applied to all columns.
                                    Defaults to None.
            chunk_size (int, optional): Specify the number of rows in each batch to be written 
                                        at a time. By default, all rows will be written at once.
                                        Defaults to None.
        """
        df.columns = [x.upper() for x in df.columns]
        self._logger.info(f'Begining upload to {table_name}')
        df.to_sql(table_name, self.engine.connect(), index=False, chunksize=chunk_size,
                  method=pd_writer, if_exists=if_exists)
        self._logger.info(f'Dataframe uploaded to {table_name}')

    def test_connection(self):
        """
        Tests a connection for snowflake from instantiated object
        """
        with self.engine.connect() as connection:
            try:
                if connection:
                    self._logger.info("connection to snowflake successful")
                else:
                    self._logger.info("connection to snowflake failed")
            finally:
                self.engine.dispose()
                connection.close()
        pass

# How to Use

## Start Snowflake Connection

In [ ]:
show_doc(SnowflakeTool)

<h2 id="SnowflakeTool" class="doc_header"><code>class</code> <code>SnowflakeTool</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>SnowflakeTool</code>(**`sfAccount`**:`str`=*`None`*, **`sfUser`**:`str`=*`None`*, **`sfPswd`**:`str`=*`None`*, **`sfWarehouse`**:`str`=*`None`*, **`sfDatabase`**:`str`=*`None`*, **`sfSchema`**:`str`=*`None`*, **`sfRole`**:`str`=*`None`*, **`delimiter`**:`str`=*`';'`*, **`logger`**=*`None`*)

Class that holds basic snowflake functionality including testing connection
and running queries.

In [ ]:
sf = SnowflakeTool(sfAccount=os.environ['sfAccount'],
                   sfUser=os.environ['sfUser'],
                   sfPswd=os.environ['sfPswd'],
                   sfWarehouse=os.environ['sfWarehouse'],
                   sfDatabase=os.environ['sfDatabase'],
                   sfSchema=os.environ['sfSchema'],
                   sfRole=os.environ['sfRole'])
assert sf.test_connection() == None, 'anything else the connection has failed'

## Pandas to Snowflake

In [ ]:
#hide
from easymigration.utils.parseyaml import *
from easymigration.imports import *
sf.run_str_query('DROP TABLE IF EXISTS dsdequerytest;')
__file__ = './'
dict1 = [{'ecid': 150, 'home': 'CA', 'avg_visits': 0.20, 'LTR': 6},
         {'ecid': 151, 'home': 'LA', 'avg_visits': 10, 'LTR': 2},
         {'ecid': 160, 'home': 'CO', 'avg_visits': 0.56, 'LTR': 4},
         {'ecid': 100, 'home': 'LA', 'avg_visits': 2.0, 'LTR': 3}]
df = pd.DataFrame(dict1)
yaml = ParseYaml(os.path.join(os.path.dirname(__file__), 'files/snowflake.yaml')).get_yaml(['test'])

In [ ]:
show_doc(SnowflakeTool.infer_to_snowflake)

<h4 id="SnowflakeTool.infer_to_snowflake" class="doc_header"><code>SnowflakeTool.infer_to_snowflake</code><a href="__main__.py#L222" class="source_link" style="float:right">[source]</a></h4>

> <code>SnowflakeTool.infer_to_snowflake</code>(**`df`**:`DataFrame`, **`table_name`**:`str`, **`if_exists`**:`str`=*`'fail'`*, **`dtype`**:`dict`=*`None`*, **`chunk_size`**:`int`=*`None`*)

Args:
    df (pd.DataFrame): The dataframe that is going to snowflake
    table_name (str): Table name that will be created in snowflake
    if_exists (str, optional): How to behave if the table already exists.
                                * fail: Raise a ValueError.
                                * replace: Drop the table before inserting new values.
                                * append: Insert new values to the existing table.
                                Defaults to 'fail'.
    dtype (dict, optional): Specifying the datatype for columns. If a dictionary is used, the
                            keys should be the column names and the values should be the
                            SQLAlchemy types or strings for the sqlite3 legacy mode. If a
                            scalar is provided, it will be applied to all columns.
                            Defaults to None.
    chunk_size (int, optional): Specify the number of rows in each batch to be written 
                                at a time. By default, all rows will be written at once.
                                Defaults to None.

In [ ]:
sf.pandas_to_snowflake(df=df,
                        user=os.environ['sfUser'],
                        password=os.environ['sfPswd'],
                        account=os.environ['sfAccount'],
                        warehouse=os.environ['sfWarehouse'],
                        database=os.environ['sfDatabase'],
                        schema=os.environ['sfSchema'],
                        table_name='dsdequerytest',
                        chunk_size = None,
                        create_table= True,
                        create_statement= yaml.get('create_query_table'))
df = sf.run_str_query("SELECT * FROM easymigrationtest")
assert df.shape == (4, 4), 'Query 4 observations w/ 4 columns from easymigrationtest'

In [ ]:
show_doc(SnowflakeTool.infer_to_snowflake)

<h4 id="SnowflakeTool.infer_to_snowflake" class="doc_header"><code>SnowflakeTool.infer_to_snowflake</code><a href="__main__.py#L222" class="source_link" style="float:right">[source]</a></h4>

> <code>SnowflakeTool.infer_to_snowflake</code>(**`df`**:`DataFrame`, **`table_name`**:`str`, **`if_exists`**:`str`=*`'fail'`*, **`dtype`**:`dict`=*`None`*, **`chunk_size`**:`int`=*`None`*)

Args:
    df (pd.DataFrame): The dataframe that is going to snowflake
    table_name (str): Table name that will be created in snowflake
    if_exists (str, optional): How to behave if the table already exists.
                                * fail: Raise a ValueError.
                                * replace: Drop the table before inserting new values.
                                * append: Insert new values to the existing table.
                                Defaults to 'fail'.
    dtype (dict, optional): Specifying the datatype for columns. If a dictionary is used, the
                            keys should be the column names and the values should be the
                            SQLAlchemy types or strings for the sqlite3 legacy mode. If a
                            scalar is provided, it will be applied to all columns.
                            Defaults to None.
    chunk_size (int, optional): Specify the number of rows in each batch to be written 
                                at a time. By default, all rows will be written at once.
                                Defaults to None.

In [ ]:
sf.run_str_query('DROP TABLE IF EXISTS easymigrationtest;')

In [ ]:
sf.infer_to_snowflake(df,
                      table_name='easymigrationtest',
                      if_exists='append')
# sf.run_str_query("SELECT * FROM dsdequerytest")
df = sf.run_str_query("SELECT * FROM easymigrationtest"); df.head()

In [ ]:
assert df.shape == (4, 4), 'Query 4 observations w/ 4 columns from easymigrationtest'
sf.run_str_query('DROP TABLE IF EXISTS easymigrationtest;')

## SQL Execution

The point of this function is to allow a user to make create a table/View and then query the table with a specific command. This can be done the same way with ``execute_file_query``, but the final command will be a the end of the file.

In [ ]:
show_doc(SnowflakeTool.execute_file)

<h4 id="SnowflakeTool.execute_file" class="doc_header"><code>SnowflakeTool.execute_file</code><a href="__main__.py#L124" class="source_link" style="float:right">[source]</a></h4>

> <code>SnowflakeTool.execute_file</code>(**`sql`**:`str`=*`None`*, **`query_path`**:`str`=*`None`*)

will run sql file or txt file that is seperated by self.delimenter
with the option to add a custom sql command if there is something you
want to add to the file being executed

Args:
    sql (str, optional): Optional Command to add on top of the file being executed. Defaults to None.
    query_path (str, optional): location to sql/txt file to execute. Defaults to None.

Returns:
    pd.DataFrame: returns query results

In [ ]:
df = sf.execute_file(sql='SELECT * FROM easymigrationtest', query_path='testing/view_test.txt')
assert df.shape == (10, 25), 'Query 10 observations w/ 25 columns from easymigrationtest'
sf.run_str_query('DROP TABLE IF EXISTS easymigrationtest;')

This is very similar to the above function, but only allows a file to be passed.

In [ ]:
show_doc(SnowflakeTool.execute_file_query)

<h4 id="SnowflakeTool.execute_file_query" class="doc_header"><code>SnowflakeTool.execute_file_query</code><a href="__main__.py#L99" class="source_link" style="float:right">[source]</a></h4>

> <code>SnowflakeTool.execute_file_query</code>(**`query_path`**:`str`)

Same as ``excecute file``, but with no sql option
might not need this.

Args:
    query_path (str): file location to execute

In [ ]:
df = sf.execute_file_query('testing/view_test.txt')
sf.run_str_query('DROP TABLE IF EXISTS dsde_test;')

In [ ]:
show_doc(SnowflakeTool.run_str_query)

<h4 id="SnowflakeTool.run_str_query" class="doc_header"><code>SnowflakeTool.run_str_query</code><a href="__main__.py#L73" class="source_link" style="float:right">[source]</a></h4>

> <code>SnowflakeTool.run_str_query</code>(**`query`**:`str`)

This function will connect to snowflake and run a query that is passed
in as a string.
Args:
    query (str): SQL Query String
Returns:
    pd.DataFrame/ None: dependent on the query string

In [ ]:
df = sf.run_str_query("SELECT * \
                       FROM easymigrationtest \
                       LIMIT 10;")
assert df.shape == (10, 25), 'Query 10 observations w/ 25 columns from easymigrationtest'